In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install -q --upgrade keras-nlp
!pip install -q --upgrade keras  # Upgrade to Keras 3.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.4 which is incompatible.


In [3]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # or "tensorflow" or "torch"

import keras_nlp
import keras

# améliore la rapidité d'entrainement
keras.mixed_precision.set_global_policy("mixed_float16")

In [4]:
import pandas as pd
# from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras_nlp.models import GPT2CausalLMPreprocessor, GPT2CausalLM
from tensorflow.keras.models import load_model, save_model


# encoding permet de gerer les problèmes dues à certain caracteres
# error_bad_lins permet de gerer les lignes qui sont mal formatés
# nrows correspond au nombre de ligne qu'on recupere depuis le csv
# Ne pas oublier denlever le paramètre nrows = 100 pour les vrais tests
text_df = pd.read_csv('/content/drive/MyDrive/Corona_NLP_train.csv', encoding="iso-8859-1", error_bad_lines=False,nrows=100)
text_df['OriginalTweet'] = text_df['OriginalTweet'].astype(str)
text = list(text_df['OriginalTweet'].values)


<ipython-input-4-e4d663703a85>:12: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  text_df = pd.read_csv('/content/drive/MyDrive/Corona_NLP_train.csv', encoding="iso-8859-1", error_bad_lines=False,nrows=100)


In [5]:
# initialisation du processeur de gpt-2 (code du prof)

preprocessor = GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=128,
)

gpt2_lm = GPT2CausalLM.from_preset(
    "gpt2_base_en",
    preprocessor=preprocessor
)

gpt2_lm.summary()


100%|██████████| 448/448 [00:00<00:00, 286kB/s]
100%|██████████| 446k/446k [00:00<00:00, 1.26MB/s]
100%|██████████| 0.99M/0.99M [00:00<00:00, 2.41MB/s]
100%|██████████| 484/484 [00:00<00:00, 1.17MB/s]
100%|██████████| 475M/475M [00:22<00:00, 21.8MB/s]


Preprocessor: "gpt2_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gpt2_tokenizer (GPT2Tokenizer)                     │                                              50,257 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gpt2_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃     Param # ┃ Connected to                   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │           0 │ -                              │
├───────────────────────────────┼───────────────────────────┼─────────────┼────────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │           0 │ -                              │
├───────────────────────────────┼───────────────────────────┼─────────────┼────────────────────────────────┤
│ gpt2_backbone (GPT2Backbone)  │ (None, None, 768)         │ 124,439,808 │ padding_mask[0][0],            │
│                               │                           │             │ token_ids[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────┼────────────────────────────────┤
│ token_embedding               │ (None, None, 50257)       │  38,597,376 │ gpt2_backbone[0][0]            │
│ (ReversibleEmbedding)         │                           │             │                                │
└───────────────────────────────┴───────────────────────────┴─────────────┴────────────────────────────────┘

 Total params: 124,439,808 (474.70 MB)

 Trainable params: 124,439,808 (474.70 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Compiplation du model
# gpt2_lm.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

# entraine le model gpt-2 qu'on a avec nos données
# hesitez pas à changer le epochs à 1/2 juste pour voir ce que ça fait
# Attention : on mettra le epochs à 10 plus tards pour que la réponses sois plus
# précise
gpt2_lm.fit(text, epochs=1, batch_size=32)


4/4 ━━━━━━━━━━━━━━━━━━━━ 219s 42s/step - loss: 2.8636 - sparse_categorical_accuracy: 0.1196


In [ ]:
from tensorflow.keras.models import save_model

# On sauvegarde le model pour éviter de relancer tout à chaque fois
save_model(gpt2_lm, '/content/drive/MyDrive/my_model.keras')  # Extension .keras

In [ ]:
# Si le modèle à été sauvegarder on a juste à le réutiliser
from tensorflow.keras.models import load_model, save_model

gpt2_lm = load_model('/content/drive/MyDrive/my_model.keras')

In [7]:
import numpy as np


# model gpt-2 de base
seed_text = "what is covid-19 ?"
generated_text = gpt2_lm.generate(seed_text, max_length=50)
print("\nGPT-2 output:")
print(generated_text)




GPT-2 output:
what is covid-19 ?

What is covidian-19 ?

what is covid-19.

Why is it called covid ?

What is covid-19.

What is covid


In [8]:
text_test = pd.read_csv('/content/drive/MyDrive/Corona_NLP_test.csv', encoding="iso-8859-1", error_bad_lines=False,nrows=100)
text_test['OriginalTweet'] = text_df['OriginalTweet'].astype(str)
text_tt = list(text_test['OriginalTweet'].values)

<ipython-input-8-a47717b2503a>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  text_test = pd.read_csv('/content/drive/MyDrive/Corona_NLP_test.csv', encoding="iso-8859-1", error_bad_lines=False,nrows=100)


In [9]:
accuracy = gpt2_lm.evaluate(text_tt, text_tt)

4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - loss: 2.5582 - sparse_categorical_accuracy: 0.1429
